In [5]:
from tqdm.notebook import tqdm
from result_saver import SaverProvider
provider = SaverProvider() 

In [6]:
from Scratch import metadata_loader

df = metadata_loader(True)
df = df[~df['job_status'].isin(['JobStatus.DONE', 'JobStatus.ERROR', 'JobStatus.CANCELLED'])]
df = df[:1000]
# df = df[df['meas_level']==0]
df

,creation_date,notebook_name,backend_name,job_id,job_name,job_metadata,tags,meas_level,init_qubits,meas_return,...,optimization_level,code,distance,rounds,logical,layout,descr,num_CNOTS_layers,num_CNOT_layers,resets
2637,2024-03-11 08:44:24.856000+01:00,get_IQ_calibration_data_Sherbrooke,ibm_sherbrooke,cqqbcjar43fg008x59m0,NaN,{},[Calibration],1.0,True,single,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
from time import sleep

OVERWRITE = False
IGNORE_RUNNING = False

start_idx = 0 # Initialize starting index

while True:  # Outer loop to restart when a specific error occurs
    try:
        for idx, job_id in enumerate(tqdm(df.job_id[start_idx:])):

            provider.retrieve_job(job_id, _ignore_running=IGNORE_RUNNING, overwrite=OVERWRITE)

            start_idx = start_idx + 1  
        break  
    except KeyboardInterrupt:
        print('Interrupted')
        break  
    except Exception as e:
        if 'RequestsApiError' in str(type(e)):
            print(f'\nRequestsApiError occurred at job index {start_idx}, retrying...')
            sleep(5) 
        else:
            print(f'An unexpected error occurred: {e}')
            break  
            # sleep(5)


  0%|          | 0/1 [00:00<?, ?it/s]

19:20:59 Warning: Job ID cqqbcjar43fg008x59m0 not found in /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77


In [50]:
from qiskit_ibm_provider import IBMProvider
ibm_provider = IBMProvider()

job = ibm_provider.retrieve_job('cqpymbs2b6ng008y69e0')

In [55]:
job.result().date

IBMJobFailureError: 'Job failed: Failed to publish job: maximum payload exceeded'

# update the metadata with the execution date

In [12]:
from Scratch import update_metadata

df = metadata_loader(_extract=True)
df = df[~df['job_status'].isin(['JobStatus.ERROR', 'JobStatus.CANCELLED'])]

while True:  # Outer loop to restart when a specific error occurs
    try:
        for idx, job_id in enumerate(tqdm(df.job_id[start_idx:])):

            ibm_job = provider.retrieve_job(job_id, _ignore_running=False)
            additional_dict = {"execution_date": str(ibm_job.result().date)}
            update_metadata(ibm_job, "None_Backend", additional_dict)

            start_idx = start_idx + 1  
        break  
    except KeyboardInterrupt:
        print('Interrupted')
        break  
    except Exception as e:
        if 'RequestsApiError' in str(type(e)):
            print(f'\nRequestsApiError occurred at job index {start_idx}, retrying...')
            sleep(5) 
        else:
            print(f'An unexpected error occurred: {e}')
            break  
            

  0%|          | 0/1179 [00:00<?, ?it/s]